In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import random
import datetime as dt

from job_client_graphql import Client, CellDisplay

## Prerequisite
+ Launch JOB server - see [documentation](TBD) section **5-Examples**

In [4]:
pages = {
    'hello-world': 'https://observablehq.com/@oscar6echo/job-example-hello-world',
}

In [5]:
s = Client(server_url='http://localhost:4001/graphql',
           default_channel='channel-1',
           password='mysecret',
           pages=pages,         
           debug=False)
print(s.build_url('hello-world'))
s.show_btn_page('hello-world')

https://observablehq.com/@oscar6echo/job-example-hello-world?host=localhost&port=4001&path=/graphql&ssl=False&channel=channel-1


## write

In [6]:
def rnd_data():
    return ''.join(random.choices('abcdefghijk', k=10)) + str(random.randint(0,1e6))

In [7]:
name, value = 'toto', rnd_data()
print(name, value)
s.write(name, value)

toto gagddgbaed102429


True

In [8]:
name, value = 'titi', rnd_data()
print(name, value)
s.write(name, value)

titi dcdjefeebk692305


True

In [9]:
name, value = 'tata', rnd_data()
print(name, value)
s.write(name, value, expiry=10) # self delete after 10s

tata kajfgakebb702851


True

## publish

In [10]:
name, value = 'tutu', rnd_data() 
print(name, value)
s.publish(name, value)

tutu jgkggfdhdj579229


True

## publishWrite

In [11]:
name, value = 'tete', rnd_data()
print(name, value)
s.publish_write(name, value, expiry=30) # self delete after 30s

tete ajaciifacc609845


True

## read

In [12]:
name = 'toto'
s.read(name)

178

In [13]:
name = 'titi'
s.read(name)

190.78421352427543

In [14]:
name = 'tata'
s.read(name)

'kajfgakebb702851'

## readHisto

In [15]:
name = 'toto'
s.read_histo(name, 4, timestamp=True)

[{'value': 101,
  'timestamp': datetime.datetime(2020, 5, 15, 20, 36, 34, 523000)},
 {'value': 178,
  'timestamp': datetime.datetime(2020, 5, 15, 20, 36, 33, 522000)},
 {'value': 113,
  'timestamp': datetime.datetime(2020, 5, 15, 20, 36, 32, 521000)},
 {'value': 180.23736876582916,
  'timestamp': datetime.datetime(2020, 5, 15, 20, 36, 30, 518000)}]

In [16]:
name = 'titi'
s.read_histo(name, 4, timestamp=False)

[{'a': 194,
  'b': 195.9181407938578,
  'c': 'iTgqhA5Flh',
  'd': {'aa': 100, 'bb': 168.53776507169334, 'cc': '8A76fYfZWh'}},
 190.78421352427543,
 'dcdjefeebk692305',
 'bZQGAbVLIV']

## Store

Client.store contains the latest values read

In [17]:
s.store

{'channel-1': {'toto': 178,
  'titi': 190.78421352427543,
  'tata': 'kajfgakebb702851'}}

Client.update contains the sequence of updates (channel, name, value) - starting from latest values read


In [18]:
s.update[:3]

[('channel-1', 'toto', 178),
 ('channel-1', 'titi', 190.78421352427543),
 ('channel-1', 'tata', 'kajfgakebb702851')]

## Subscribe

### DISPLAY
In this demo (channel:name->value) are displayed upon udpate

In [19]:
out = CellDisplay(name='subscribe-status', max_lines=5)
# show_lines = []

def show(channel, name, value):
    ts = dt.datetime.now().strftime('%H:%M:%S.%f')[:-4]
    s = f'received on {channel}: {name} --> {value} ({ts})'
    out.update(s, kind='text', append=True)

received on channel-1: toto --> 190 (22:36:59.55)
received on channel-1: titi --> {"a":143,"b":126.23748485593643,"c":"27ZwdME1HE","d":{"aa":128,"bb":129.21043022274924,"cc":"bR0nVRf9jP"}} (22:37:00.55)
received on channel-1: toto --> 124.84400979394891 (22:37:01.55)
received on channel-1: titi --> 165.86974438589897 (22:37:02.55)
received on channel-1: toto --> 163.46322841990116 (22:37:03.55)

In [20]:
out.display()

received on channel-1: toto --> 190 (22:36:59.55)
received on channel-1: titi --> {"a":143,"b":126.23748485593643,"c":"27ZwdME1HE","d":{"aa":128,"bb":129.21043022274924,"cc":"bR0nVRf9jP"}} (22:37:00.55)
received on channel-1: toto --> 124.84400979394891 (22:37:01.55)
received on channel-1: titi --> 165.86974438589897 (22:37:02.55)
received on channel-1: toto --> 163.46322841990116 (22:37:03.55)

### START
If no channel is specified the Client.default_channel is used.

In [21]:
ref = s.subscribe(action=show)

### STOP

In [22]:
s.subscribe_stop(ref)

subscription ref=TUWkAL stopped


In [24]:
out.clear()